In [1]:
import numpy as np

# Generate data (true model is Cp = a0 + a1*T + a2*T^2)
# CP in cal/mol-K and T in K
a0 = 6.190
a1 = 0.02923
a2 = -0.0000007052
theta = np.array([a0, a1, a2])

# 3 sets of experiments at T = 300K, T = 600K, and T = 900K
n = 10
N = 3 * n
x1 = np.linspace(300, 300, n)
x2 = np.linspace(600, 600, n)
x3 = np.linspace(900, 900, n)
x = np.concatenate((x1, x2, x3))

# Generate true outputs
np.random.seed(0)
y1 = a0 + a1 * x1 + a2 * x1**2
y2 = a0 + a1 * x2 + a2 * x2**2
y3 = a0 + a1 * x3 + a2 * x3**2
ytrue = np.concatenate((y1, y2, y3))

# Add random noise to true outputs
sigma = 0.1
y = ytrue + np.random.normal(0, sigma, N)

# Solve linear estimation problem and get exact param covariance
xsq = x**2
X = np.column_stack((np.ones(N), x, xsq))
Y = y




In [2]:
# Solve estimation problem
thetaest = np.linalg.inv(X.T @ X) @ X.T @ Y

# Get model predictions
Yhat = X @ thetaest

# Get residuals
e = Y - Yhat

# Obtain estimate of noise variance
sig2 = np.var(e)

# Get exact covariance matrix for parameters
C = np.linalg.inv(X.T @ X) * sig2

# Obtain exact covariance with Monte Carlo
K = 100  # Repeat estimation 100 times
th = np.zeros((3, K))

for k in range(K):
    # Perturb the output data with random noise
    yp = y + np.random.normal(0, np.sqrt(sig2), N)
    Y = yp

    # Solve estimation problem
    th[:, k] = np.linalg.inv(X.T @ X) @ X.T @ Y

# Sample covariance
thmean = np.mean(th, axis=1)
Cest = np.zeros((3, 3))

for k in range(K):
    Cest += np.outer(th[:, k] - thmean, th[:, k] - thmean)

Cest /= K

# Compare with exact
print("Exact covariance matrix C:\n", C)
print("Estimated covariance matrix Cest:\n", Cest)

Exact covariance matrix C:
 [[ 2.12686229e-02 -7.83580843e-05  6.21889558e-08]
 [-7.83580843e-05  3.04725883e-07 -2.48755823e-10]
 [ 6.21889558e-08 -2.48755823e-10  2.07296519e-13]]
Estimated covariance matrix Cest:
 [[ 2.20238515e-02 -8.19726939e-05  6.57228733e-08]
 [-8.19726939e-05  3.18257767e-07 -2.61092770e-10]
 [ 6.57228733e-08 -2.61092770e-10  2.18094656e-13]]
